<a href="https://colab.research.google.com/github/yeqinghuang516/UCSD-ECE285-Object-Detection-Using-Deep-Learning/blob/master/YOLO/YOLOv3_Train_Sageband.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Git clone the repository and install the requirments

In [0]:
! git clone https://github.com/yeqinghuang516/UCSD-ECE285-Object-Detection-Using-Deep-Learning.git
% cd UCSD-ECE285-Object-Detection-Using-Deep-Learning/YOLO
! sudo pip3 install -r requirements.txt

In [0]:
from __future__ import division

import os
import sys
import time
import datetime
import argparse
sys.path.append('/content/UCSD-ECE285-Object-Detection-Using-Deep-Learning/YOLO/')


from models import *
from utils.utils import *
from utils.logger import *
from utils.datasets import *
from mytest import evaluate

from terminaltables import AsciiTable

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim


import torchvision as tv
import numpy as np
import matplotlib.pyplot as plt
import cv2

## Setting up parameters for training

In [0]:
epochs = 200 # number of epochs
batch_size = 32 #size of each image batch
gradient_accumulations = 2 #number of gradient accums before step
model_def = 'config/yolov3.cfg' #path to model definition file
n_cpu = 8 #number of cpu threads to use during batch generation
img_size = 224#size of each image dimension
checkpoint_interval = 1 # interval between saving model weights
evaluation_interval = 2 # interval evaluations on validation set
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger = Logger("logs")
os.makedirs("output", exist_ok=True)
os.makedirs("checkpoints", exist_ok=True)
metrics = ['grid_size', 'loss', 'x', 'y', 'w', 'h', 'conf', 'cls', 'cls_acc', 'recall50', 'recall75', 'precision', 'conf_obj', 'conf_noobj']
class_names = ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']

## Download and Initialize datasets

In [0]:
os.makedirs('data', exist_ok = True)
root = 'data'
VOCdataset = VOCDataset(root, image_set = 'train', train = True)
VOCtestset = VOCDataset(root, image_set = 'val')

## Define pretrained weights (if training from previous checkpoints or pretrained weights)

In [0]:
pretrained_weights = ''

## Initiate model

In [6]:
model = Darknet(model_def, loss_mode = "modified").to(device)
model.apply(weights_init_normal);
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
cur_epoch = 0

# If specified we start from checkpoint, load checkpoint
if pretrained_weights:
    if pretrained_weights.endswith(".pth"):
        checkpoint = torch.load(pretrained_weights)
        model.load_state_dict(checkpoint['net'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        cur_epoch = checkpoint['epoch'] + 1
        print('load state dict')

The loss mode is modified


## Initiate Dataloader

In [0]:
dataloader = torch.utils.data.DataLoader(
    dataset = VOCdataset,
    batch_size= batch_size,
    num_workers= n_cpu,
    shuffle = True,
    pin_memory= True,
    drop_last = False,
    collate_fn= VOCdataset.collate_fn,
)

## Start training

In [0]:
for epoch in range(cur_epoch, epochs):
    model.train()
    start_time = time.time()
    for batch_i, (imgs, targets) in enumerate(dataloader):
      batches_done = len(dataloader) * epoch + batch_i
      imgs = Variable(imgs.to(device))
      targets = Variable(targets.to(device), requires_grad=False)

      loss, outputs = model(imgs, targets)
      loss.backward()

      if batches_done % gradient_accumulations:
          # Accumulates gradient before each step
          optimizer.step()
          optimizer.zero_grad()
    
      # ----------------
      #   Log progress
      # ----------------

      log_str = "\n---- [Epoch %d/%d, Batch %d/%d] ----\n" % (epoch, epochs, batch_i, len(dataloader))

      metric_table = [["Metrics", *[f"YOLO Layer {i}" for i in range(len(model.yolo_layers))]]]

      # Log metrics at each YOLO layer
      for i, metric in enumerate(metrics):
          formats = {m: "%.6f" for m in metrics}
          formats["grid_size"] = "%2d"
          formats["cls_acc"] = "%.2f%%"
          row_metrics = [formats[metric] % yolo.metrics.get(metric, 0) for yolo in model.yolo_layers]
          metric_table += [[metric, *row_metrics]]

          # Tensorboard logging
          tensorboard_log = []
          for j, yolo in enumerate(model.yolo_layers):
              for name, metric in yolo.metrics.items():
                  if name != "grid_size":
                      tensorboard_log += [(f"{name}_{j+1}", metric)]
          tensorboard_log += [("loss", loss.item())]
          logger.list_of_scalars_summary(tensorboard_log, batches_done)

      log_str += AsciiTable(metric_table).table
      log_str += f"\nTotal loss {loss.item()}"

      # Determine approximate time left for epoch
      epoch_batches_left = len(dataloader) - (batch_i + 1)
      time_left = datetime.timedelta(seconds=epoch_batches_left * (time.time() - start_time) / (batch_i + 1))
      log_str += f"\n---- ETA {time_left}"
      
      # print training log
      print(log_str)

      model.seen += imgs.size(0)
    
    if epoch % evaluation_interval == 0:
        print("\n---- Evaluating Model ----")
        # Evaluate the model on the validation set
        precision, recall, AP, f1, ap_class = evaluate(
            model,
            dataset = VOCtestset,
            iou_thres=0.5,
            conf_thres=0.5,
            nms_thres=0.5,
            img_size=img_size,
            batch_size=8,
        )
        evaluation_metrics = [
            ("val_precision", precision.mean()),
            ("val_recall", recall.mean()),
            ("val_mAP", AP.mean()),
            ("val_f1", f1.mean()),
        ]
        logger.list_of_scalars_summary(evaluation_metrics, epoch)

        # Print class APs and mAP
        ap_table = [["Index", "Class name", "AP"]]
        for i, c in enumerate(ap_class):
            ap_table += [[c, class_names[c], "%.5f" % AP[i]]]
        print(AsciiTable(ap_table).table)
        print(f"---- mAP {AP.mean()}")
    
    # save checkpoints
    if epoch % checkpoint_interval == 0:
        state_dict = {'net': model.state_dict(), 'optimizer': optimizer.state_dict(),'epoch': epoch}
        torch.save(state_dict, f"checkpoints/yolov3_ckpt_%d.pth" % epoch)